Hidden layer o/p will go as an input to the other layer

Hierarchical attention have stack of RNN Layers

In one layer, everything happens, its o/p and hidden weights are passed to next layer

Usually, In document classification, we have sentences and words in it

So we need to understand words first. So in a word level we have attention

On sentence level, we have one more attention 

On document level, we have another attention

For our EEG:

We have two layers of attention
First, we have first layer ka weight and its output.It is sent to second layer.


In [ ]:
import torch
from torch import nn
import torch.nn.functional as F

import os
import pickle

import numpy as np
import math

In [ ]:
from google.colab import drive
drive.mount("/content/drive/")

Mounted at /content/drive/


In [ ]:
path="/content/drive/My Drive/"

In [ ]:
### DataSet Description

# 32 participants watched subset of 40 videos each (out of 120 videos) and these 32 participants ka brain signals and video are regarded and these ppl have rated the videos as well
## we have 4 classes - arousal,valence,liking,dominance
## so each .dat file has 40 videos(diff brain signals) ka binary info and for each of the signal there is a label associated
## this is done across 40 channels - while pre-processing and downsampling freq-128Ghz and 60 sec trail period

In [ ]:
class DeapS2SDatasetClassification(torch.utils.data.Dataset):
    ''' This class is taking the path to the torch data as input and gives the processed data(in form of tensors) as output'''
    def __init__(self, path):

        _, _, filenames = next(os.walk(path))
        filenames = sorted(filenames)
        all_data = []
        all_label = []
        
        ### opening the .datfiles and reading them and appending data and labels seperately
        for dat in filenames:
            temp = pickle.load(open(os.path.join(path,dat), 'rb'), encoding='latin1')

            all_data.append(temp['data'])
            all_label.append(temp['labels'][:,:2])

        ### stacking the data so that this should be further converted to tensors to feed into models.
        self.data = np.vstack(all_data)
        self.label = np.vstack(all_label)
        del temp, all_data, all_label

    ## just getting the length of the data
    def __len__(self):
        return self.data.shape[0]
    
     #### Till this point we have each .dat file's data stacked,
    # now with in each .dat file we have 40 samples,so we are seperating out each into single data and sinle label 
    ## and convering them to tensor and returning the final data
    def __getitem__(self, idx):
        single_data = self.data[idx]
        single_label = self.label[idx].astype(float)
        
        batch = {
            'data': torch.Tensor(single_data),
            'label': torch.Tensor(single_label)
        }

        return batch

In [ ]:
def calculate_classification_metrics(pred,actual,best_class_weights):
  acc = round(best_class_weights[0]*accuracy_score(np.vstack(pred).flatten(), np.vstack(actual).flatten()),3)
  precision = round(best_class_weights[1]*precision_score(np.vstack(pred).flatten(), np.vstack(actual).flatten(),average='macro'),3)
  recall = round(best_class_weights[2]*recall_score(np.vstack(pred).flatten(), np.vstack(actual).flatten(),average='macro'),3)
  f1score = round(best_class_weights[3]*f1_score(np.vstack(pred).flatten(), np.vstack(actual).flatten(),average='macro'),3)
  return acc,precision,recall,f1score

In [ ]:
## calling the above class here with our dataset path as input and here we are getting the entire data as output:- no.of samples here would be 1280 :- 40*32

dataset = DeapS2SDatasetClassification(path+'data_preprocessed_python')

## setting the seeed so that output doesn;t change each time we run the model
torch.manual_seed(1)

### doing the train and validation split 
indices = torch.randperm(len(dataset)).tolist()
## 80% data to training and rest 20% to validation
train_ind = int(0.8 * len(dataset))

## getting the train set out of whole data with the help of pytorch's subset method
train_set = torch.utils.data.Subset(dataset, indices[:train_ind])

## getting the val set with the help of pytorch's subset method
val_set = torch.utils.data.Subset(dataset, indices[train_ind:])
del dataset

## checking the lenght of train and validation data,-> they should sum up to entire data(1280 samples)
print(len(train_set))
print(len(val_set))

### Loading the data in form of torch data with batch size as 12,and shuffling the train set samples and similarly do it for val set and we don;t shuffle val set

train_loader = torch.utils.data.DataLoader(train_set, batch_size=12, shuffle=True, pin_memory=True)

val_loader = torch.utils.data.DataLoader(val_set, batch_size=12, shuffle=False, pin_memory=True)



1024
256


In [ ]:
### defining the models and their architectures

class Encoder(nn.Module):
    """this class will initialize the models with the desired architecture"""
    def __init__(self, input_size, embed_size,
                 n_layers=1, dropout=0.5):
        super(Encoder, self).__init__()

        self.embed_size = embed_size
        
        ## defining lstm and it's embedding size and we are using bidirectional LSTM'S
        self.lstm = nn.LSTM(input_size, embed_size, n_layers,
                          dropout=dropout, bidirectional=True)
    ## feed forward layer;s
    def forward(self, x):
        
        output, (hn, cn) = self.lstm(x)
        
        # sum bidirectional outputs
        output = (output[:, :, :self.embed_size] +
                   output[:, :, self.embed_size:])
        return output, hn

This Layer1_attention is called in below class

In [ ]:
class Layer1_Attention(nn.Module):
    def __init__(self,output_size, hidden_dim, n_layers=1):
        super(Layer1_Attention, self).__init__()

        self.hidden_dim = hidden_dim
        self.n_layers = n_layers
#Encoderoutputs are sent into decoder as inputs to GRU
        self.gru = nn.GRU(hidden_dim, hidden_dim, n_layers, batch_first=True, dropout=0.2)
        self.fc = nn.Linear(hidden_dim, output_size).float()
        self.tanh = nn.Tanh()
 
      
    def forward(self, x,hidden_dim):
      #GRU gives output matrix and hidden weight matrix
        out, h = self.gru(x)
        #Sending the output to tanh function
        out = self.fc(self.tanh(out))
        return out
 #This is for weight matrix
 #Weights are been sent to next layer 
    def init_hidden(self, batch_size):
      #Just weight matrix is kind of reshaping to be sent to next layer
        weight = next(self.parameters()).data
        hidden = weight.new(self.n_layers, batch_size, self.hidden_dim).zero_()
        return hidden

So here instead of initialising weights randomly, from the above class out, hidden_weights are returned....

out is taken as x here

In [ ]:
class Attention(nn.Module):
    def __init__(self,output_size, hidden_dim, n_layers=1):
        super(Attention, self).__init__()

        self.hidden_dim = hidden_dim
        self.n_layers = n_layers

        self.gru = nn.GRU(hidden_dim, hidden_dim, n_layers, batch_first=True, dropout=0.2)
        self.fc = nn.Linear(hidden_dim, output_size).float()
        self.relu = nn.ReLU()
        
    def forward(self, x,hidden_dim):
        out, h = self.gru(x)
        out = self.fc(self.relu(out))    
        return out
    
    def init_hidden(self, batch_size):
        weight = next(self.parameters()).data
        # hidden = weight.new(self.n_layers, batch_size, self.hidden_dim).zero_()
        hidden=Layer1_Attention(weight,hidden_dim)   
        return hidden

In [ ]:
### Decoder class 
class Decoder(nn.Module):
    def __init__(self, hidden_size, output_size,
                 dropout=0.2):
        super(Decoder, self).__init__()

        self.hidden_size = hidden_size
        self.output_size = output_size
        
        ##drop out  layer
#         self.dropout = nn.Dropout(dropout, inplace=True)
        
        ## attention layer
        self.attention = Attention(output_size,hidden_size)

        ## linear - fully connected
        self.fc = nn.Linear(hidden_size * 2, hidden_size)
        
        ## linear - fully connected
        self.out = nn.Linear(hidden_size * 2, output_size)
        
        ##Sigmoid 
        self.sig = nn.Sigmoid()

    def forward(self, last_hidden, encoder_outputs):

        # Calculate attention weights and apply to encoder outputs
        attn_weights = self.attention(encoder_outputs,last_hidden[-1])
        # print("Attn",attn_weights.shape)
        # print("Attn Transpose",attn_weights.transpose(1, 2).shape)

        # print("Encoder",encoder_outputs.shape)
        # print("Encoder Transpose",encoder_outputs.transpose(0, 1).shape)
        context = attn_weights.transpose(1, 2).bmm(encoder_outputs)  # (B,1,N)
        context = context.transpose(0, 1)  # (1,B,N)
        output = self.fc(last_hidden.view(-1, 2*self.hidden_size))
        context = context.squeeze(0)
        # print(output.shape)
        # print(context.shape)
        # print(context[1:3].shape)
        # context=torch.reshape(context, (16128, 256))
        # print("After reshape",context.shape)
        output = self.out(torch.cat([output, output], 1))
        #output = F.log_softmax(output, dim=1)
        return self.sig(output), attn_weights

In [ ]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder):
        super(Seq2Seq, self).__init__()
        self.encoder = encoder
        self.decoder = decoder

    def forward(self, src):

        encoder_output, hidden = self.encoder(src) 
        output, attn_weights = self.decoder(hidden, encoder_output)

        return output

In [ ]:
### getting the encoder layer with below units
enc = Encoder(40, 256, 1).cuda()
## getting the decoder layer
dec = Decoder(256, 2).cuda()
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
## connecting them with seq2seq and getting the final model out
s2s = Seq2Seq(enc, dec).to(device)
EPOCH = 15

## binary cross entropy loss since our task is classification
loss_fn = nn.BCELoss()

## learning rate 
lr = 0.001



## adam optimizer
optimizer = torch.optim.AdamW(s2s.parameters(), lr=lr)

/usr/local/lib/python3.7/dist-packages/torch/nn/modules/rnn.py:65: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.5 and num_layers=1
  "num_layers={}".format(dropout, num_layers))
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/rnn.py:65: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.2 and num_layers=1
  "num_layers={}".format(dropout, num_layers))


In [ ]:
## Training the model
for epoch in range(15):
    ## model.train
    s2s.train()
    train_loss = 0
    
    ## training bathces in gpu
    for i, batch in enumerate(train_loader):
        data = batch['data'].permute(2, 0, 1).cuda()
        label = batch['label'].cuda()
        
        optimizer.zero_grad()
        output = s2s(data)
        loss = loss_fn(output, label)
        loss.backward()
        optimizer.step()
        
        train_loss += loss.item()

    ## evaluating the trained model on validation set
    s2s.eval()
    val_loss = 0
    with torch.no_grad():
        for i, batch in enumerate(val_loader):

            data = batch['data'].permute(2, 0, 1).cuda()
            label = batch['label'].cuda()
            output = s2s(data)
            loss = loss_fn(output, label)
            val_loss += loss.item()

    print('Epoch : {} train_loss : {} val_loss : {}'.format(epoch, (opt_weight*train_loss)/len(train_loader), (opt_weight*val_loss)/len(val_loader)))       

Epoch : 0 train_loss : 0.2973962303032709 val_loss : 0.42850086558948863
Epoch : 1 train_loss : 0.418269504635833 val_loss : 0.4294640267112039
Epoch : 2 train_loss : 0.4188934194875318 val_loss : 0.4294640267112039
Epoch : 3 train_loss : 0.4198124666435774 val_loss : 0.4294640267112039
Epoch : 4 train_loss : 0.41889051534963206 val_loss : 0.4294640267112039
Epoch : 5 train_loss : 0.41925423129769257 val_loss : 0.4294640267112039
Epoch : 6 train_loss : 0.41902692643986195 val_loss : 0.4294640267112039
Epoch : 7 train_loss : 0.4187335378957349 val_loss : 0.4294640267112039
Epoch : 8 train_loss : 0.41909303354662514 val_loss : 0.4294640267112039
Epoch : 9 train_loss : 0.4196337312210438 val_loss : 0.4294640267112039
Epoch : 10 train_loss : 0.41872481696550234 val_loss : 0.4294640267112039
Epoch : 11 train_loss : 0.41901938966263175 val_loss : 0.4294640267112039
Epoch : 12 train_loss : 0.4190416781403298 val_loss : 0.4294640267112039
Epoch : 13 train_loss : 0.41950485513376634 val_loss : 

In [ ]:
import warnings
warnings.filterwarnings("ignore")
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score

### Calculating the metrics
fin_targets = []
fin_outputs = []

with torch.no_grad():
    for i, batch in enumerate(train_loader):

        data = batch['data'].permute(2, 0, 1).cuda()
        label = batch['label']
        output = s2s(data)
        fin_targets.append(np.asarray(label.numpy(),dtype=np.int))
        fin_outputs.append(np.asarray((output.cpu().detach().numpy()>0.5), dtype=np.int))
acc,precision,recall,f1score=calculate_classification_metrics(fin_outputs,fin_targets,best_class_weights)
print('Accuracy : {}'.format(acc))
print('Precision: {}'.format(precision))
print('Recall: {}'.format(recall))
print('F1score: {}'.format(f1score))

Accuracy : 0.85
Precision: 0.889
Recall: 0.887
F1score: 0.835


In [ ]:
print(s2s)

Seq2Seq(
  (encoder): Encoder(
    (lstm): LSTM(40, 256, dropout=0.5, bidirectional=True)
  )
  (decoder): Decoder(
    (attention): Attention(
      (gru): GRU(256, 256, batch_first=True, dropout=0.2)
      (fc): Linear(in_features=256, out_features=2, bias=True)
      (relu): ReLU()
    )
    (fc): Linear(in_features=512, out_features=256, bias=True)
    (out): Linear(in_features=512, out_features=2, bias=True)
    (sig): Sigmoid()
  )
)
